# 🎮 DIVAnd game

The aim of this notebook is to choose the location of 100 observations to
optimally reconstruct the provided sea surface temperature anomaly using DIVAnd.
We use the _Reynolds et al. 2002_ [OI SST](https://www.psl.noaa.gov/data/gridded/data.noaa.oisst.v2.html) for the month January and remove the zonal average. This is assumed to be the "true field" in the followed.

1. You should work in groups. Choose a group name and adapt the `groupname` accordingly
2. ⚠️ Please change only the marked code (after "Your turn !" and before "Do not make any change below")
3. You choose:
     * the location of 100 observations (the vectors `xobs` and `yobs` which corresponds to longitude in degrees East and latitude in degrees North)
     * the correlation length `len`
     * the uncertainty of the observation `epsilon2`
4. You are allowed to guide your choice by using the true field `v` (the target variable)
5. Your results must be reproducible (the locations of the observations are saved in a text file at the end of the notebook)
6. You can submit as many test as you wish, only the best result with lowest root mean square (RMS) error is shown.


## First load all modules

In [ ]:
import Pkg
Pkg.activate("../..")
Pkg.instantiate()
using DIVAnd
using Makie, CairoMakie, GeoMakie
using NCDatasets
using Statistics
using Random
using Downloads: download
using Interpolations
using DelimitedFiles
include("../config.jl")

## Download data file if necessary

In [ ]:
filename = sstfile
download_check(sstfile, sstfileURL)

## Load the data

In [ ]:
ds = NCDataset(filename)
vfull = reverse(nomissing(ds["sst"][:, :, 1], NaN), dims = 2)
lon = ds["lon"][:]
lat = reverse(ds["lat"][:])
close(ds);

## Prepare inputs
### Create land-sea mask

The array `mask` is 1 on sea and 0 on land.

In [ ]:
mask = isfinite.(vfull);

### Remove the zonal average

In [ ]:
v0 = copy(vfull)
v0[.!mask] .= 0
v = vfull .- sum(v0, dims = 1) ./ sum(mask, dims = 1)
sz = size(v)
xi, yi = DIVAnd.ndgrid(lon, lat);

### Scale factors (inverse of the resolution)

In [ ]:
pm = ones(sz) ./ ((xi[2, 1] - xi[1, 1]) .* cosd.(yi));
pn = ones(sz) / (yi[1, 2] - yi[1, 1]);

### Number of observations

In [ ]:
nobs = 100

### Noise on observations [°C]

In [ ]:
noise = 0.5

## Your turn !

You can change the following code and variables

In [ ]:
# your group name (better use only ASCII symbols)
groupname = "Just-a-test"
# Correlation length in arc degrees (1° ~ 111 km)
len = 20
# Uncertainty of the observation [adimensional]
epsilon2 = 0.001
# Choose 100 random location (only sea points)
# Is there a better way?
indexobs = shuffle(findall(mask[:] .== 1))[1:nobs];
xobs = xi[indexobs];
yobs = yi[indexobs];

## 🚫 Do not make any change below

Extract the pseudo-observations

In [ ]:
xobs = mod.(xobs, 360)
itp = interpolate((lon, lat), v, Gridded(Constant()))
vobs = itp.(xobs, yobs);

Add some noise

In [ ]:
Random.seed!(42)
vobs_perturbed = vobs + noise * randn(nobs);

Make the analysis

In [ ]:
vi, s = DIVAndrun(mask, (pm, pn), (xi, yi), (xobs, yobs), vobs_perturbed, len, epsilon2);

### Plot the results

In [ ]:
function map(ax; cl = extrema(filter(isfinite, v)))
    xlims!(ax, extrema(lon))
    ylims!(ax, extrema(lat))
    contourf!(ax, lon, lat, mask, levels = [0, 0.5], colormap = Reverse(:Greys))
end

fig = Figure()

ax1 = Axis(fig[1, 1], title = "True field")
hm1 = heatmap!(ax1, lon, lat, v, colorrange = extrema(filter(isfinite, v)));
Colorbar(fig[2, 1], hm1, vertical = false)
map(ax1)

ax2 = Axis(fig[1, 2], title = "Observations")
sc1 = scatter!(ax2, xi[indexobs], yi[indexobs], color = vobs, markersize = 10)
Colorbar(fig[2, 2], sc1, vertical = false)
map(ax2)

ax3 = Axis(fig[3, 1], title = "Analysis field")
hm2 = heatmap!(ax3, lon, lat, vi, colorrange = extrema(filter(isfinite, v)))
Colorbar(fig[4, 1], hm2, vertical = false)
map(ax3)

ax4 = Axis(fig[3, 2], title = "Analysis - true field")
hm3 = heatmap!(ax4, lon, lat, vi - v, colormap = :RdBu, colorrange = [-3.0, 3.0])
Colorbar(fig[4, 2], hm3, vertical = false)
map(ax4)

display(fig)

RMS difference between analysis and true field

In [ ]:
RMS_difference = sqrt(mean(filter(isfinite, (vi - v) .^ 2)))

A function to submit the RMS error to score board:
http://data-assimilation.net/scores/?game=DIVAnd2024

In [ ]:
function submit_results(groupname, rms_velocity)
    function myescape(str)
        for (s1, s2) in Dict("{" => "%7B", "}" => "%7D", "\"" => "%22", " " => "%20")
            str = replace(str, s1 => s2)
        end
        return str
    end
    myjson(data) = "{" * join(["\"$k\":$(string(v))" for (k, v) in data], ",") * "}"
    game = "DIVAnd2024"

    data_dict = Dict()
    #=
    data = JSON.json(data_dict)
    groupname_esc = URIParser.escape(groupname)
    data_esc = URIParser.escape(data)
    =#
    data = myjson(data_dict)
    groupname_esc = myescape(groupname)
    data_esc = myescape(data)
    baseurl = "http://data-assimilation.net/scores"
    URL = "$(baseurl)/new?game=$(game)&name=$(groupname_esc)&value=$(rms_velocity)&data=$(data_esc)"
    rm(download(URL))
    println("Check scores at: $(baseurl)/?game=$(game)\n")
end

In [ ]:
submit_results(groupname, RMS_difference)

In [ ]:
filename = "positions-rms-RMS-$RMS_difference-len-$len-epsilon2-$epsilon2.txt"
writedlm(filename, [xobs yobs])